<a href="https://colab.research.google.com/github/sugarforever/LangChain-Tutorials/blob/main/OpenAI_Chat_Completions_16k.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# OpenAI new chat model gpt-3.5-turbo-16k

gpt-3.5-turbo-16k offers 4 times the context length of gpt-3.5-turbo at twice the price: $0.003/1K input tokens and $0.004/1K output tokens.

16k context means the model can now support ~20 pages of text in a single request.

In [31]:
!pip install tiktoken langchain openai --quiet --upgrade

In [32]:
import os
import openai

openai.api_key = "Your OpenAI API Key"

In [33]:
!wget https://uniswap.org/whitepaper-v3.pdf

--2023-06-15 21:20:37--  https://uniswap.org/whitepaper-v3.pdf
Resolving uniswap.org (uniswap.org)... 104.18.23.54, 104.18.22.54, 2606:4700::6812:1736, ...
Connecting to uniswap.org (uniswap.org)|104.18.23.54|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1500865 (1.4M) [application/pdf]
Saving to: ‘whitepaper-v3.pdf.1’

whitepaper-v3.pdf.1 100%[===================>]   1.43M  --.-KB/s    in 0.02s   

2023-06-15 21:20:37 (65.6 MB/s) - ‘whitepaper-v3.pdf.1’ saved [1500865/1500865]



In [34]:
!ls -alt whitepaper-v3.pdf

-rw-r--r-- 1 root root 1500865 Jun 15 20:28 whitepaper-v3.pdf


In [35]:
import tiktoken

encoding = tiktoken.encoding_for_model('gpt-3.5-turbo-16k')
encoding

<Encoding 'cl100k_base'>

In [36]:
encoding.encode("tiktoken is great!")

[83, 1609, 5963, 374, 2294, 0]

In [ ]:
!pip install PyPDF2 pymupdf --quiet

In [37]:
from langchain.document_loaders import PyMuPDFLoader

pages = PyMuPDFLoader('whitepaper-v3.pdf').load()

all_content = ''
for page in pages:
  all_content += page.page_content + "\n"

In [38]:
print(len(all_content))

41711


In [39]:
tokens = encoding.encode(all_content)
len(tokens)

12377

In [44]:
completion = openai.ChatCompletion.create(
  model="gpt-3.5-turbo-16k",
  messages = [
    {"role": "system", "content" : "You are a chatbot which can answer questions based on the paper user supplies."},
    {"role": "user", "content" : "I have the following whitepaper of uniswap v3"},
    {"role": "user", "content" : all_content},
    {"role": "user", "content" : "What is liquidity oracle?"}
  ]
)
print(completion)

{
  "id": "chatcmpl-7Rope0Wps0WHHCnQDAw1ZIZzQbWtM",
  "object": "chat.completion",
  "created": 1686864610,
  "model": "gpt-3.5-turbo-16k-0613",
  "choices": [
    {
      "index": 0,
      "message": {
        "role": "assistant",
        "content": "A liquidity oracle is a mechanism that provides information about the available liquidity in a particular market or trading pair. In the context of decentralized exchanges (DEXs) like Uniswap, a liquidity oracle provides data on the depth and volume of liquidity available for different token pairs.\n\nThe liquidity oracle collects and aggregates data from various liquidity sources, such as liquidity providers and market makers, to calculate the overall liquidity in a specific trading pair. It helps traders and users of DEXs to make informed decisions by providing insights into the liquidity landscape.\n\nThe data provided by a liquidity oracle can be used in various ways, including calculating slippage for trades, determining price impact

In [42]:
completion['choices'][0]['message']['content']

'A liquidity oracle is a mechanism or system that provides information about the liquidity available in a specific market or trading pair. It helps users and smart contracts determine the depth and availability of liquidity for trading or other purposes.\n\nIn the context of Uniswap v3, the liquidity oracle is a time-weighted average liquidity oracle. It provides information about the average liquidity for a given pair of tokens over a specific period of time. It allows users to query the recent liquidity accumulator values, eliminating the need to checkpoint the accumulator value at the exact beginning and end of the period for which a time-weighted average liquidity is being measured.\n\nThe liquidity oracle provides valuable information for various purposes, such as optimizing trading strategies, determining slippage, or making decisions related to liquidity provisioning and mining programs.'